# Update Databricks Secrets Access Control Lists
***

The purpose of this notebook is to update the access control lists for already defined Databricks Secret Scopes.  

In order to run this notebook, the *run_as* user must have MANAGE permission on the Secret Scope.  

> DANGER:  Did not share your personal Secret Scopes (typically based on your user name if following practices outlined in the <a href="$./Set Databricks Secrets with Python SDK">Set Databricks Secrets with Python SDK</a> notebook found in this same repo) as that will allow other user's to invoke all of your secrets including secrets such as tokens that are likely unique to you.  Only change the ACLs of Secret Scopes that should be used by many users.  

To change the ACL using this notebook you'll need three input parameters:  

* **secret_scope**: the Databricks Secret Scope whose ACL you want to change.  
* **principal**: the Service Principal, group, or user name that should be added to the ACL.  Note that *users* is the group role for *All workspace users*.  
* **permission**:  One of the available access control list permissions: MANAGE, WRITE, READ. The difference between MANAGE and WRITE is that with MANAGE the **principal** is able to alter ACLs as well.  Note that the *admins* group always has MANAGE for all scopes.  Use the **permissison** REVOKE to remove the **principal** from the ACL.  Note: if you revoke your own access (such as removing the only group you're in with MANAGE, or your specific username as the creator of the scope), then only an admin or someone else with MANAGE can add you back.

In [0]:
%pip install databricks-sdk --upgrade

In [0]:
dbutils.library.restartPython()

In [0]:
dbutils.widgets.text("secret_scope", "", "Databricks Secret Scope")
dbutils.widgets.text("principal", "", "Service Principal, Group, or User")
dbutils.widgets.dropdown("permission", "READ", ["READ", "WRITE", "MANAGE", "REVOKE"], "ACL Permission")

In [0]:
from databricks.sdk import WorkspaceClient
from databricks.sdk.service import workspace

w = WorkspaceClient()

In [0]:
secret_scope = dbutils.widgets.get("secret_scope")
principal = dbutils.widgets.get("principal")
permission = dbutils.widgets.get("permission")

print(
f"""
   secret_scope: {secret_scope}
   principal: {principal}
   permission: {permission}   
""")

In [0]:
scopes = w.secrets.list_scopes()
scopes = [scope.as_dict() for scope in scopes]
scope_exists = any(scope['name'] == secret_scope for scope in scopes)
print(f"scope_exists: {scope_exists}")

if scope_exists == True:
  acls = w.secrets.list_acls(scope=secret_scope)
  acls = [acl.as_dict() for acl in acls]
  print(acls)
else:
  raise Exception("Secret Scope does not exist, please check the input widget or create a new secret scope with the 'Set Databricks Secrets with Python SDK' notebook.")

In [0]:
if permission == "REVOKE":
    for acl in acls: 
        if acl['principal'] == principal: 
            w.secrets.delete_acl(scope=secret_scope, principal=acl['principal'])
            print(f"Revoking access to Databricks Secret Scope '{secret_scope}' for '{principal}'.")
        else:
            pass
else:
    w.secrets.put_acl(
        scope = secret_scope
        ,principal = principal
        ,permission = workspace.AclPermission[permission]
    )
    print(f"Granting `{permission}` access to Databricks Secret Scope '{secret_scope}' for '{principal}'.")

In [0]:
acls = w.secrets.list_acls(scope=secret_scope)
acls = [acl.as_dict() for acl in acls]
print(acls)